In [0]:
import numpy as np
import networkx as nx

In [0]:
class Nodo:
    def __init__(self,nodo, puzzle=None,nodoshijos=None, accion=None):
        self.nodo=nodo
        self.nodoPadre=None
        self.nodoshijos=nodoshijos
        self.puzzle=puzzle
        self.accion = accion
    def __str__(self):
        return self.nodo
    def __eq__(self, obj):
        return (self.puzzle==obj.puzzle).all()
    def obtenerlistaordenadaheuristica(self, nodometa):
        for i in range(len(self.nodoshijos)):
          for j in range(len(self.nodoshijos)): 
            vi = self.calculateHeuristic(self.nodoshijos[i], nodometa)
            vj = self.calculateHeuristic(self.nodoshijos[j], nodometa)
            if(vj < vi):
                nodotemp = self.nodoshijos[i]
                self.nodoshijos[i] = self.nodoshijos[j]
                self.nodoshijos[j] = nodotemp
    def calculateHeuristic(self,nodei, nodom):
      ret = np.array(nodei.puzzle) == np.array(nodom.puzzle)
      return np.count_nonzero(ret == False)

In [0]:
class PuzzleClass:
        
    def buscar_espacio_vacio(self,arreglo):
      for i in range(arreglo.shape[0]):
        for j in range(arreglo.shape[1]):
          if(arreglo[i,j] == 0):
            return np.array([i,j])
        
    def pos_adyacente(self, i, j): 
      i_arriba = i - 1 
      i_abajo = i + 1  
      j_izquierda = j - 1
      j_derecha= j + 1    
      return np.array([[i_arriba,j, "Arriba"],[i_abajo,j, "Abajo"],[i, j_izquierda, "Izquierda"], [i, j_derecha, "Derecha"]])
  
    def crear_puzzle_hijos(self,i,j,pos_ayacentes,puzzle):
        puzzle_hijos= []
        fil,col = puzzle.shape
        puzzle_acciones= []
        for i_ady1,j_ady1, accion in pos_ayacentes:     
            i_ady = int(i_ady1)
            j_ady = int(j_ady1)
            puzzle_hijo = puzzle.copy()
            val_vacio = puzzle[i,j]        
            if (0<=i_ady and i_ady<fil) and (0<=j_ady and j_ady<col):
                val_temp = puzzle[i_ady,j_ady]
                puzzle_hijo[i,j] = val_temp
                puzzle_hijo[i_ady,j_ady] = val_vacio
                puzzle_hijos.append(puzzle_hijo)    
                puzzle_acciones.append(accion)        
        return puzzle_hijos, puzzle_acciones   
    
    def crear_nodos_hijos(self,nodo):
        (i,j) = self.buscar_espacio_vacio(nodo.puzzle)
        pos_ayacentes = self.pos_adyacente(i, j)
        puzzle_hijos, puzzle_accion = self.crear_puzzle_hijos(i,j,pos_ayacentes,nodo.puzzle)
        nodos_hijos=[]
        id_nodo = "A_"
        con = 0
        for hijo in puzzle_hijos:
            id_nodo_hijo = id_nodo+str(con)
            nodos_hijos.append(Nodo(id_nodo_hijo,hijo, accion=puzzle_accion[con]))
            con = con+1
        return nodos_hijos
    
    def es_nodo_meta(self,nodo,nodoMeta):
        return (nodo.puzzle==nodoMeta.puzzle).all()
    
    def resolver_puzzle(self,nodoInicial,nodoMeta):
        nodosAbiertos=[]
        nodosVisitados=[]    
        nodosAbiertos.append(nodoInicial)  
        c = 0
        while nodosAbiertos:
            nodo=nodosAbiertos.pop(0)   
            #calculateHeuristic(nodo)                 
            if nodo==nodoMeta:
                print('META')
                print(nodo.accion)
                return nodo,True
            if not nodo in nodosVisitados: 
                nodosVisitados.append(nodo)            
                print(nodo.puzzle)
                print(nodo.accion)
                c = c + 1
                print("Nodo: ", c)
                print("----------------------")
                nodo.nodoshijos=self.crear_nodos_hijos(nodo)    
                nodo.obtenerlistaordenadaheuristica(nodoMeta)        
                for nodohijo in nodo.nodoshijos:
                    if not nodohijo in nodosVisitados:                                                         
                        if not nodohijo.nodoPadre:
                            nodohijo.nodoPadre = nodo                         
                        nodosAbiertos.insert(0, nodohijo)
        return None, False

In [72]:
PuzzleClass = PuzzleClass()
puzzle = np.array([[2,8,3], [1,6,4],[7,0,5]])
nodo = Nodo("A",puzzle)
puzzleM = np.array([[1,2,3], [8,0,4],[7,6,5]])
nodoMeta = Nodo("G",puzzleM)
nodoMeta,estado = PuzzleClass.resolver_puzzle(nodo,nodoMeta)
print(nodoMeta.puzzle)

[[2 8 3]
 [1 6 4]
 [7 0 5]]
None
Nodo:  1
----------------------
[[2 8 3]
 [1 0 4]
 [7 6 5]]
Arriba
Nodo:  2
----------------------
[[2 0 3]
 [1 8 4]
 [7 6 5]]
Arriba
Nodo:  3
----------------------
[[0 2 3]
 [1 8 4]
 [7 6 5]]
Izquierda
Nodo:  4
----------------------
[[1 2 3]
 [0 8 4]
 [7 6 5]]
Abajo
Nodo:  5
----------------------
META
Derecha
[[1 2 3]
 [8 0 4]
 [7 6 5]]
